In [46]:
import numpy as np
import pandas as pd
from radioReduction import *
import matplotlib.pyplot as plt
from scipy.stats import norm
import random
%matplotlib tk

In [2]:
dt = pd.read_table('1314/2M1314e15-LLRR.dat')

In [30]:
binned = binning(dt,sec=10)
binned.plot.scatter('secs','re',yerr='ure',color='black')
plt.show()

In [58]:
def model(p,time):
    '''
    Linear model fitting to the data
    '''
    m = p[0]
    b = p[1]
    y = m*time+b
    return y

def chi2(p,data,time,error):
    chi2 = np.sum((data-model(p,time))**2/error**2)
    return chi2

def mcmc(p_int,**kwargs):
    '''
    Does an mcmc walk using Metropolis on data, finding the best fit
    parameters to a linear line
    Assume parameters are Gaussian
    '''
    steps = kwargs.get('steps',1000)
    step_size = kwargs.get('step_size',1)
    data = kwargs.get('data')
    time = kwargs.get('time')
    error = kwargs.get('error')
    
    # Setting up list to hold the parameters
    p = p_int
    m = []
    b = []
    accept = []

    # Burning in 10% of the steps
    for i in range(int(.1*steps)):
        p_prime = [norm(p[0],step_size).rvs(),norm(p[1],step_size).rvs()]
        prob = chi2(p_prime,data,time,error)/chi2(p,data,time,error)

        if prob > 1:
            p = p_prime
        else:
            rand = random.uniform(0,1)
            if rand <= prob:
                p = p_prime
            else:
                p_prime = 0
                
    # Doing the actual MCMC
    for i in range(steps):
        p_prime = [norm(p[0],step_size).rvs(),norm(p[1],step_size).rvs()]
        prob = chi2(p_prime,data,time,error)/chi2(p,data,time,error)

        # If prob is more than 1, accept it 
        if prob > 1:
            p = p_prime
            m.append(p[0])
            b.append(p[1])
            accept.append(1)

        # If it is less than 1, do a probability of acceptance
        else:
            rand = random.uniform(0,1)
            if rand <= prob:
                p = p_prime
                m.append(p[0])
                b.append(p[1])
                accept.append(1)
            else:
                p_prime = 0
                accept.append(0)

    # Making things arrays
    p_vals = np.array([m,b])
    accept = np.array(accept)
    
    return p_vals,accept

In [77]:
p_vals, accept = mcmc([2,1],steps=10**4,step_size=.02,data=binned['re'],time=binned['secs'],error=binned['ure'])

In [78]:
m = p_vals[0];b=p_vals[1]

h = plt.figure(1)
plt.hist(m,color='black',bins=100,histtype='step')
plt.hist(b,color='red',bins=100,histtype='step')
h.show()

h2 = plt.figure(2)
plt.hist(accept,color='black')
h2.show()